In [1]:
import datetime
import time
import requests

import json
import os
import pandas as pd
import numpy as np

#threading
import threading
import timeit

from email.mime.text import MIMEText
import smtplib

#declare global lock object
global lock
lock = threading.Lock()

In [2]:
def symbol_downloader(symbol, directory, days=60, days_ago=0):
    dt = datetime.datetime.now()
    UnixTime = int(time.mktime(dt.timetuple()))
    #web variables
    url = 'https://query1.finance.yahoo.com/v8/finance/chart/'+symbol+'?period1='+str(UnixTime-86400*(days+days_ago))+'&period2='+str(UnixTime-86400*days_ago)+'&interval=1d&indicators=quote%7Csma~60&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-CA&region=CA&corsDomain=ca.finance.yahoo.com'
    #proxies
    http_proxy  = ''
    https_proxy = ''
    ftp_proxy   = ''
    proxyDict = { 
                  "http"  : http_proxy, 
                  "https" : https_proxy, 
                  "ftp"   : ftp_proxy
                }

    hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
       'Connection': 'keep-alive'}
    
    try:
        resp = requests.get(url, headers=hdr)
        if resp.status_code != 200:
            # This means something went wrong.
            print(resp.status_code)
        data = json.loads(resp.text)
        #json unpack
        timestamp = data['chart']['result'][0]['timestamp']
        timestamp = [datetime.datetime.fromtimestamp(x).strftime('%Y%m%d') for x in timestamp]
        quote = data['chart']['result'][0]['indicators']['quote'][0]
        sma50 = data['chart']['result'][0]['indicators']['sma'][0]['sma']
        stock_df = pd.DataFrame(quote)
        #index is symbol and timestamp
        stock_df.index = [str(x) for x in timestamp]

        #moving averages
        stock_df['sma'] = sma50
        stock_df['vol20'] = stock_df['volume'].rolling(window=20).mean()
        #predict on yesterday's averages
        stock_df['sma'] = stock_df['sma'].shift(1)
        stock_df['vol20'] = stock_df['vol20'].shift(1)
        stock_df = stock_df.dropna()

        #derived columns
#         stock_df['close'] = stock_df['close']/stock_df['sma']
#         stock_df['high'] = stock_df['high']/stock_df['sma']
#         stock_df['low'] = stock_df['low']/stock_df['sma']
#         stock_df['open'] = stock_df['open']/stock_df['sma']
        stock_df['volume'] = stock_df['volume']/stock_df['vol20']

        #stock_df.drop(['close', 'high', 'low', 'open','volume','sma','vol20'], axis=1, inplace=True)
        stock_df = stock_df.dropna()
        stock_df = stock_df[~(stock_df == np.inf).any(axis=1)]
        if len(stock_df) != 0:
            stock_df.to_csv('{}{}.csv'.format(directory,symbol[:4]))
        
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        raise SystemExit(e)

In [ ]:
if __name__ == '__main__':

    #start timer
    start = timeit.default_timer()

    symbolslist = open('symbols_nasdaq.txt').read().split('\n')

    directory = './data_nasdaq/'

    threadlist = []

    for u in symbolslist:

        t = threading.Thread(target = symbol_downloader,args=(u,directory))
        t.start()
        threadlist.append(t)
        #sets top limit of active threads to 50
        while threading.activeCount()>50:
            a=0
        #print threading.activeCount()

    for b in threadlist:
        b.join()
    print ("# of threads: ", len(threadlist))


Exception in thread Thread-12:
Traceback (most recent call last):
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4d2a03930e50>", line 27, in symbol_downloader
    timestamp = data['chart']['result'][0]['timestamp']
KeyError: 'timestamp'

Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4d2a03930e50>", line 27, in symbol_downloader
    timestamp = data['chart']['result'][0]['timestamp']
TypeError: 'NoneType' object is not subscriptable



404
404
404


Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4d2a03930e50>", line 27, in symbol_downloader
    timestamp = data['chart']['result'][0]['timestamp']
TypeError: 'NoneType' object is not subscriptable

Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4d2a03930e50>", line 27, in symbol_downloader
    timestamp = data['chart']['result'][0]['timestamp']
KeyError: 'timestamp'
Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\Richard\Anaconda3\lib\threa

404
404


Exception in thread Thread-64:
Traceback (most recent call last):
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4d2a03930e50>", line 27, in symbol_downloader
    timestamp = data['chart']['result'][0]['timestamp']
TypeError: 'NoneType' object is not subscriptable

Exception in thread Thread-67:
Traceback (most recent call last):
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4d2a03930e50>", line 27, in symbol_downloader
    timestamp = data['chart']['result'][0]['timestamp']
TypeError: 'NoneType' object is not subscriptable

Exception in thread Thread-68:
Traceback (most recent call last):
  File "C:\Use

...

Exception in thread Thread-3415:
Traceback (most recent call last):
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4d2a03930e50>", line 27, in symbol_downloader
    timestamp = data['chart']['result'][0]['timestamp']
TypeError: 'NoneType' object is not subscriptable

Exception in thread Thread-3417:
Traceback (most recent call last):
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4d2a03930e50>", line 27, in symbol_downloader
    timestamp = data['chart']['result'][0]['timestamp']
TypeError: 'NoneType' object is not subscriptable



404


Exception in thread Thread-3424:
Traceback (most recent call last):
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4d2a03930e50>", line 27, in symbol_downloader
    timestamp = data['chart']['result'][0]['timestamp']
TypeError: 'NoneType' object is not subscriptable



404


Exception in thread Thread-3426:
Traceback (most recent call last):
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Richard\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4d2a03930e50>", line 27, in symbol_downloader
    timestamp = data['chart']['result'][0]['timestamp']
TypeError: 'NoneType' object is not subscriptable



In [5]:
#Analyze
to_send = ''
directory = './data_nasdaq/'
# get list with filenames in folder and throw away all non ncsv
files = [f for f in os.listdir(directory) if f.endswith('.csv')]
for f in files:
    try:
        stock_df = pd.read_csv(os.path.join(directory, f))

        for i in range(int((len(stock_df)*0.5))): #engulfing candel pattern looping through all available data 
            i += 1 #start range from 1 instead of 0
            if float(stock_df['close'].iloc[-(i+1)]) < float(stock_df['open'].iloc[-(i+1)]) \
                and float(stock_df['close'].iloc[-i]) > float(stock_df['open'].iloc[-(i+1)]) \
                and float(stock_df['open'].iloc[-i]) < float(stock_df['close'].iloc[-(i+1)]) \
                and (float(stock_df['close'].iloc[-i])/float(stock_df['sma'].iloc[-i])) >= 1 \
                and float(stock_df['close'].iloc[-i]) <= 20 and float(stock_df['close'].iloc[-i]) >= 2 \
                and float(stock_df['volume'].iloc[-i]) >= 3:

                to_send += '{} has engulfing candle with high volume on {}, and is under $20 \n'.format(f[:-4],(stock_df['Unnamed: 0'].iloc[-i]))
                print ('{} has engulfing candle with high volume on {}, and is under $20 \n'.format(f[:-4],(stock_df['Unnamed: 0'].iloc[-i])))
                
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        print (f)
        raise SystemExit(e)

A.csv
AA.csv
AAAR.csv
AABA.csv
AACA.csv
AAGH.csv
AAGI.csv
AAIG.csv
AAL.csv
AAMC.csv
AAME.csv
AAN..csv
AAN.csv
AAOI.csv
AAON.csv
AAP.csv
AAPL.csv
AASZ.csv
AAT.csv
AATP.csv
AAU.csv
AAUK.csv
AAV..csv
AAWW.csv
AAXJ.csv
AAXN.csv
AB.csv
ABB.csv
ABBV.csv
ABC.csv
ABCB.csv
ABDC.csv
ABEO.csv
ABEV.csv
ABG.csv
ABGS.csv
ABI..csv
ABIO.csv
ABM.csv
ABMD.csv
ABN..csv
ABR-.csv
ABR..csv
ABR.csv
ABSC.csv
ABT..csv
ABT.csv
ABTX.csv
ABUS.csv
ABVC.csv
ABX..csv
ABZP.csv
AC.csv
AC.T.csv
ACA.csv
ACAD.csv
ACAM.csv
ACB.csv
ACBI.csv
ACBM.csv
ACC.csv
ACCO.csv
ACCY.csv
ACD..csv
ACDS.csv
ACDV.csv
ACEH.csv
ACFF.csv
ACFN.csv
ACGB.csv
ACGL.csv
ACH.csv
ACHC.csv
ACIA.csv
ACIU.csv
ACIW.csv
ACKD.csv
ACLS.csv
ACM.csv
ACMR.csv
ACN.csv
ACNB.csv
ACND.csv
ACOP.csv
ACOR.csv
ACP.csv
ACQ..csv
ACRE.csv
ACRG.csv
ACRS.csv
ACRX.csv
ACSA.csv
ACST.csv
ACTG.csv
ACTL.csv
ACTR.csv
ACTT.csv
ACU.csv
ACUR.csv
ACV.csv
ACWI.csv
ACWX.csv
ACY.csv
AD.T.csv
ADAP.csv
ADBC.csv
ADBE.csv
ADC.csv
ADDY.csv
ADES.csv
ADI.csv
ADK..csv
ADKC.csv
ADKT.csv
ADM.cs

In [4]:
#email output
with open('C:\\Users\\Richard\\Desktop\\Python\\hotmail.txt', 'rb') as f:
    email_list = str(f.read()).split(',')
    emailAddress = email_list[0][2:]
    password = email_list[1][:-1]

    msg = MIMEText(to_send)
    recipients = [emailAddress, 'michelleusdenski@gmail.com','Guowei88888@msn.com']#'jacob.si@outlook.com','Greggh_101@hotmail.com']
    msg['Subject'] = '%s stock analysis: potentials - %s' % (str(datetime.datetime.today().strftime('%Y-%m-%d')), str(len(to_send.split('\n'))-1))
    msg['From'] = emailAddress
    msg['To'] = ', '.join(recipients)
    try:
        s = smtplib.SMTP('smtp-mail.outlook.com', 25)
        s.ehlo()  # Hostname to send for this command defaults to the fully qualified domain name of the local host.
        s.starttls()  # Puts connection to SMTP server in TLS mode
        s.ehlo()
        s.login(emailAddress, password)
        s.sendmail(emailAddress, recipients, msg.as_string())
        s.quit()
        print ('email sent to: %s' % emailAddress)
    except:
        raise
        
#timer
stop = timeit.default_timer()
print ("seconds of operation: " , stop - start)

email sent to: richardguo0122@hotmail.com


NameError: name 'start' is not defined